In [1]:
import sys
sys.path.append("../")

In [2]:
import os
from dotenv import load_dotenv 
load_dotenv("../constants/.env")

True

In [3]:
api_key = os.environ.get('BINANCE_KEY')
secret_key = os.environ.get('BINANCE_SECRET')

In [4]:
from binance.client import Client
import pandas as pd

In [5]:

import time


client = Client(api_key, secret_key)



In [6]:
from collections import defaultdict





In [7]:
import itertools
from itertools import permutations



In [ ]:
import math
import decimal 

def fetch_prices_with_best(symbols):
    """Obtém os melhores preços de compra (bid) e venda (ask) para os pares fornecidos."""
    prices = {}
    for symbol in symbols:
        try:
            best_bid, best_ask = fetch_best_prices(symbol)
            prices[symbol] = {'bid': best_bid, 'ask': best_ask}
        except Exception as e:
            print(f"Erro ao buscar preços para {symbol}: {e}")
            prices[symbol] = None
    return prices
    
def cache_exchange_info():
    """Obtém e armazena localmente as informações de troca."""
    try:
        exchange_info = client.get_exchange_info()
        return exchange_info
    except Exception as e:
        print(f"Erro ao buscar informações da exchange: {e}")
        return None

# Chame esta função uma vez no início
exchange_info = cache_exchange_info()


def get_lot_size(symbol):
    """Obtém os detalhes do filtro LOT_SIZE para o par a partir do cache."""

    if not exchange_info:
        print("Informações da exchange não estão disponíveis.")
        return None
    try:
        for s in exchange_info['symbols']:
            if s['symbol'] == symbol:
                for f in s['filters']:
                    if f['filterType'] == 'LOT_SIZE':
                        return {
                            "minQty": float(f['minQty']),
                            "maxQty": float(f['maxQty']),
                            "stepSize": float(f['stepSize'])
                        }
    except Exception as e:
        print(f"Erro ao obter LOT_SIZE para {symbol}: {e}")
    return None
    
def execute_order(order_type, symbol, quantity):
    """
    Executa uma ordem de mercado.
    Args:
        order_type (str): 'BUY' ou 'SELL'.
        symbol (str): Par de negociação (ex.: 'BTCUSDT').
        quantity (float): Quantidade do ativo a negociar.
    Returns:
        dict: Detalhes da ordem executada ou None em caso de erro.
    """
    try:
        if order_type == 'BUY':
            order = client.order_market_buy(symbol=symbol, quantity=quantity)
        elif order_type == 'SELL':
            order = client.order_market_sell(symbol=symbol, quantity=quantity)
        else:
            raise ValueError("Tipo de ordem inválido.")
        print(f"Ordem executada: {order}")
        return order
    except Exception as e:
        print(f"Erro ao executar ordem {order_type} para {symbol}: {e}")
        return None


def get_balance(asset):
    """Obtém o saldo atual de um ativo."""
    try:
        account_info = client.get_account()
        balances = {item['asset']: float(item['free']) for item in account_info['balances']}
        return balances.get(asset, 0.0)
    except Exception as e:
        print(f"Erro ao obter saldo de {asset}: {e}")
        return 0.0
        
def adjust_quantity(quantity, step_size):
    """
    Ajusta a quantidade para o múltiplo mais próximo permitido pelo stepSize,
    respeitando a precisão do step_size.

    Args:
        quantity (float): Quantidade desejada.
        step_size (float): Incremento permitido pelo LOT_SIZE.

    Returns:
        float: Quantidade ajustada.
    """
    # Determinar o número de casas decimais com base no step_size
    precision = abs(decimal.Decimal(str(step_size)).as_tuple().exponent)
    
    # Ajustar a quantidade para o múltiplo permitido
    adjusted_quantity = math.floor(quantity / step_size) * step_size

    # Retornar o valor ajustado com a precisão correta
    return round(adjusted_quantity, precision)

def fetch_best_prices(symbol):
    """Obtém os melhores preços de compra (bid) e venda (ask) do livro de ordens."""
    try:
        order_book = client.get_order_book(symbol=symbol)
        best_bid = float(order_book['bids'][0][0])  # Melhor preço de compra
        best_ask = float(order_book['asks'][0][0])  # Melhor preço de venda
        return best_bid, best_ask
    except Exception as e:
        print(f"Erro ao obter preços para {symbol}: {e}")
        return None, None
        
def fetch_prices(symbols):
    """Obtém os preços dos pares fornecidos."""
    prices = {}
    for symbol in symbols:
        try:
            ticker = client.get_ticker(symbol=symbol)
            prices[symbol] = float(ticker['lastPrice'])
        except Exception as e:
            print(f"Erro ao buscar preço para {symbol}: {e}")
            prices[symbol] = None
    return prices

def triangular_arbitrage(brl_balance, steps_size, pair_prices, fee_rate=0.001):
    """Calcula lucro potencial na arbitragem triangular, ajustando pelas taxas."""

    # Etapa 1: Converter BRL para o ativo do primeiro par
    first_conversion = brl_balance / pair_prices[0]
    first_conversion *= (1 - fee_rate)
    first_conversion = adjust_quantity(first_conversion, steps_size[0])

    # Etapa 2: Converter ativo do segundo par
    second_conversion = first_conversion / pair_prices[1]
    second_conversion *= (1 - fee_rate)
    second_conversion = adjust_quantity(second_conversion, steps_size[1])

    # Etapa 3: Converter ativo do terceiro par de volta para BRL
    final_brl = second_conversion * pair_prices[2]
    final_brl *= (1 - fee_rate)
    final_brl = adjust_quantity(final_brl, steps_size[2])

    # Lucro estimado
    profit = final_brl - brl_balance
    return profit, final_brl
    
def triangular_arbitrage_best_prices(brl_balance, steps_size, pair_prices, fee_rate=0.001):
    """
    Calcula lucro potencial na arbitragem triangular usando bid/ask para preços.
    Args:
        brl_balance (float): Saldo inicial em BRL.
        steps_size (list): Step sizes para os pares.
        pair_prices (list): Lista de dicionários com preços bid/ask para os pares.
        fee_rate (float): Taxa aplicada em cada operação.

    Returns:
        tuple: Lucro estimado e saldo final.
    """
    # Etapa 1: Comprar o ativo do primeiro par (USDT com BRL)
    first_conversion = brl_balance / pair_prices[0]['ask']  # Compra (usa ask)
    first_conversion *= (1 - fee_rate)
    # first_conversion = adjust_quantity(first_conversion, 0.00000001)
    # print(f"Estou comprando {first_conversion} BTC com {brl_balance} USDT. ")
    
    # Etapa 2: Comprar o ativo do segundo par
    second_conversion = first_conversion / pair_prices[1]['ask']  # Compra (usa ask)
    second_conversion *= (1 - fee_rate)
    # second_conversion = adjust_quantity(second_conversion, 0.00000001)
    # print(f"Estou comprando {second_conversion} ENA com {first_conversion} BTC. ")
    
    # Etapa 3: Vender o ativo do terceiro par para BRL
    final_brl = second_conversion * pair_prices[2]['bid']  # Venda (usa bid)
    final_brl *= (1 - fee_rate)
    # final_brl = adjust_quantity(final_brl, 0.00000001)
    # print(f"Estou vendendo {second_conversion} ENA e obtendo {final_brl} USDT. ")
    
    # Lucro estimado
    profit = final_brl - brl_balance
    return profit, final_brl


# Lista de pares de arbitragem
arbitrage_opportunities = [
    ('USDTBRL', 'POLUSDT', 'POLBRL'),
    ('USDTBRL', 'SOLUSDT', 'SOLBRL'),
    ('USDTBRL', 'LINKUSDT', 'LINKBRL'),
    ('USDTBRL', 'XRPUSDT', 'XRPBRL'),
    ('USDTBRL', 'DOGEUSDT', 'DOGEBRL'),
    ('USDTBRL', 'NEARUSDT', 'NEARBRL'),
    ('USDTBRL', 'ADAUSDT', 'ADABRL'),
    ('USDTBRL', 'SHIBUSDT', 'SHIBBRL'),
    ('USDTBRL', 'BNBUSDT', 'BNBBRL'),
    ('USDTBRL', 'DOTUSDT', 'DOTBRL'),
    ('USDTBRL', 'PEPEUSDT', 'PEPEBRL'),
    ('USDTBRL', 'RENDERUSDT', 'RENDERBRL'),
    ('USDTBRL', 'ENAUSDT', 'ENABRL'),
    ('USDTBRL', 'CATIUSDT', 'CATIBRL'),
    ('USDTBRL', 'NOTUSDT', 'NOTBRL'),
    ('USDTBRL', 'SUIUSDT', 'SUIBRL'),
    ('USDTBRL', 'FETUSDT', 'FETBRL'),
    ('USDTBRL', 'GALAUSDT', 'GALABRL'),
    ('USDTBRL', 'NEIROUSDT', 'NEIROBRL'),
    ('USDTBRL', 'PNUTUSDT', 'PNUTBRL'),


    ('BTCUSDT', 'AAVEBTC', 'AAVEUSDT'),
    ('BTCUSDT', 'POLBTC', 'POLUSDT'),
    ('BTCUSDT', 'SOLBTC', 'SOLUSDT'),
    ('BTCUSDT', 'LINKBTC', 'LINKUSDT'),
    ('BTCUSDT', 'XRPBTC', 'XRPUSDT'),
    ('BTCUSDT', 'DOGEBTC', 'DOGEUSDT'),
    ('BTCUSDT', 'NEARBTC', 'NEARUSDT'),
    ('BTCUSDT', 'ADABTC', 'ADAUSDT'),
    ('BTCUSDT', 'BNBBTC', 'BNBUSDT'),
    ('BTCUSDT', 'DOTBTC', 'DOTUSDT'),
    ('BTCUSDT', 'RENDERBTC', 'RENDERUSDT'),
    ('BTCUSDT', 'ENABTC', 'ENAUSDT'),

    ('USDTBRL', 'ENAUSDT', 'ENABRL'),
    ('BTCUSDT', 'ENABTC', 'ENAUSDT'),
    ('BTCBRL', 'ENABTC', 'ENABRL'),
]

# Saldo inicial em BRL
brl_balance = 50

while True:
    # Itera sobre cada combinação de pares na lista
    for opportunity in arbitrage_opportunities:
        try:
            

            # Obter lot sizes para cada par
            lot_size1 = get_lot_size(opportunity[0])
            lot_size2 = get_lot_size(opportunity[1])
            lot_size3 = get_lot_size(opportunity[2])

            if not lot_size1 or not lot_size2 or not lot_size3:
                print(f"Erro: Não foi possível obter lot sizes para os pares {opportunity}.")
                break
        
            # Extrair step sizes
            steps_size = []
            steps_size.append(lot_size1['stepSize'])
            steps_size.append(lot_size2['stepSize'])
            steps_size.append(lot_size3['stepSize'])

            # Obter preços bid/ask para os pares
            prices = fetch_prices_with_best(opportunity)
            # prices = fetch_prices(opportunity)

                
            if all(prices.values()):  # Certifica-se de que todos os preços foram obtidos
                
                pair_prices = []
                pair_prices.append(prices[opportunity[0]])
                pair_prices.append(prices[opportunity[1]])
                pair_prices.append(prices[opportunity[2]])

                # Calcula o lucro potencial ajustado pelas taxas
                # profit, final_usdt = triangular_arbitrage(
                #     brl_balance, steps_size, pair_prices
                # )

                profit, final_usdt = triangular_arbitrage_best_prices(
                    brl_balance, steps_size, pair_prices
                )

                # print(f"liquidity: {check_liquidity(opportunity[0])}, {check_liquidity(opportunity[1])}, {check_liquidity(opportunity[2])}")

                
                if profit > 0:  # Se há lucro potencial
                    print(f"\nPreços atuais para {opportunity}: {prices}")
                    print(f"Lucro: ${profit:.2f}")
                    print(f"Saldo final estimado (após taxas): ${final_usdt:.2f}")
                    # Adicione a lógica para executar as ordens, se necessário.
                else:
                    print(f"Sem oportunidades para {opportunity}. Lucro: ${profit:.2f}")

            else:
                print(f"Erro ao obter preços para os pares {opportunity}.")
        except Exception as e:
            print(f"Erro ao processar a arbitragem para {opportunity}: {e}")

    # Espera antes de buscar preços novamente
    # time.sleep(1)


Sem oportunidades para ('USDTBRL', 'POLUSDT', 'POLBRL'). Lucro: $-0.20
Sem oportunidades para ('USDTBRL', 'SOLUSDT', 'SOLBRL'). Lucro: $-0.16
Sem oportunidades para ('USDTBRL', 'LINKUSDT', 'LINKBRL'). Lucro: $-0.22
Sem oportunidades para ('USDTBRL', 'XRPUSDT', 'XRPBRL'). Lucro: $-0.17
Sem oportunidades para ('USDTBRL', 'DOGEUSDT', 'DOGEBRL'). Lucro: $-0.17
Sem oportunidades para ('USDTBRL', 'NEARUSDT', 'NEARBRL'). Lucro: $-0.30
Sem oportunidades para ('USDTBRL', 'ADAUSDT', 'ADABRL'). Lucro: $-0.18
Sem oportunidades para ('USDTBRL', 'SHIBUSDT', 'SHIBBRL'). Lucro: $-0.19
Sem oportunidades para ('USDTBRL', 'BNBUSDT', 'BNBBRL'). Lucro: $-0.17
Sem oportunidades para ('USDTBRL', 'DOTUSDT', 'DOTBRL'). Lucro: $-0.19
Sem oportunidades para ('USDTBRL', 'PEPEUSDT', 'PEPEBRL'). Lucro: $-0.26
Sem oportunidades para ('USDTBRL', 'RENDERUSDT', 'RENDERBRL'). Lucro: $-0.27
Sem oportunidades para ('USDTBRL', 'ENAUSDT', 'ENABRL'). Lucro: $-0.33
Sem oportunidades para ('USDTBRL', 'CATIUSDT', 'CATIBRL'). Lu

In [45]:
value = 0.1294
new_value = value*(1-0.001)
new_value

0.12927059999999999

In [43]:
adjust_quantity(new_value, 0.01)

0.12

In [ ]:
import math
import decimal 

def fetch_prices_with_best(symbols):
    """Obtém os melhores preços de compra (bid) e venda (ask) para os pares fornecidos."""
    prices = {}
    for symbol in symbols:
        try:
            best_bid, best_ask = fetch_best_prices(symbol)
            prices[symbol] = {'bid': best_bid, 'ask': best_ask}
        except Exception as e:
            print(f"Erro ao buscar preços para {symbol}: {e}")
            prices[symbol] = None
    return prices
    
def cache_exchange_info():
    """Obtém e armazena localmente as informações de troca."""
    try:
        exchange_info = client.get_exchange_info()
        return exchange_info
    except Exception as e:
        print(f"Erro ao buscar informações da exchange: {e}")
        return None

# Chame esta função uma vez no início
exchange_info = cache_exchange_info()


def get_lot_size(symbol):
    """Obtém os detalhes do filtro LOT_SIZE para o par a partir do cache."""

    if not exchange_info:
        print("Informações da exchange não estão disponíveis.")
        return None
    try:
        for s in exchange_info['symbols']:
            if s['symbol'] == symbol:
                for f in s['filters']:
                    if f['filterType'] == 'LOT_SIZE':
                        return {
                            "minQty": float(f['minQty']),
                            "maxQty": float(f['maxQty']),
                            "stepSize": float(f['stepSize'])
                        }
    except Exception as e:
        print(f"Erro ao obter LOT_SIZE para {symbol}: {e}")
    return None
    
def execute_order(order_type, symbol, quantity):
    """
    Executa uma ordem de mercado.
    Args:
        order_type (str): 'BUY' ou 'SELL'.
        symbol (str): Par de negociação (ex.: 'BTCUSDT').
        quantity (float): Quantidade do ativo a negociar.
    Returns:
        dict: Detalhes da ordem executada ou None em caso de erro.
    """
    try:
        if order_type == 'BUY':
            order = client.order_market_buy(symbol=symbol, quantity=quantity)
        elif order_type == 'SELL':
            order = client.order_market_sell(symbol=symbol, quantity=quantity)
        else:
            raise ValueError("Tipo de ordem inválido.")
        print(f"Ordem executada: {order}")
        return order
    except Exception as e:
        print(f"Erro ao executar ordem {order_type} para {symbol}: {e}")
        return None


def get_balance(asset):
    """Obtém o saldo atual de um ativo."""
    try:
        account_info = client.get_account()
        balances = {item['asset']: float(item['free']) for item in account_info['balances']}
        return balances.get(asset, 0.0)
    except Exception as e:
        print(f"Erro ao obter saldo de {asset}: {e}")
        return 0.0
        
def adjust_quantity(quantity, step_size):
    """
    Ajusta a quantidade para o múltiplo mais próximo permitido pelo stepSize,
    respeitando a precisão do step_size.

    Args:
        quantity (float): Quantidade desejada.
        step_size (float): Incremento permitido pelo LOT_SIZE.

    Returns:
        float: Quantidade ajustada.
    """
    # Determinar o número de casas decimais com base no step_size
    precision = abs(decimal.Decimal(str(step_size)).as_tuple().exponent)
    
    # Ajustar a quantidade para o múltiplo permitido
    adjusted_quantity = math.floor(quantity / step_size) * step_size

    # Retornar o valor ajustado com a precisão correta
    return round(adjusted_quantity, precision)

def fetch_best_prices(symbol):
    """Obtém os melhores preços de compra (bid) e venda (ask) do livro de ordens."""
    try:
        order_book = client.get_order_book(symbol=symbol)
        best_bid = float(order_book['bids'][0][0])  # Melhor preço de compra
        best_ask = float(order_book['asks'][0][0])  # Melhor preço de venda
        return best_bid, best_ask
    except Exception as e:
        print(f"Erro ao obter preços para {symbol}: {e}")
        return None, None
        
def fetch_prices(symbols):
    """Obtém os preços dos pares fornecidos."""
    prices = {}
    for symbol in symbols:
        try:
            ticker = client.get_ticker(symbol=symbol)
            prices[symbol] = float(ticker['lastPrice'])
        except Exception as e:
            print(f"Erro ao buscar preço para {symbol}: {e}")
            prices[symbol] = None
    return prices

def triangular_arbitrage(brl_balance, steps_size, pair_prices, fee_rate=0.002):
    """
    Calcula lucro potencial na arbitragem triangular usando bid/ask para preços.
    Args:
        brl_balance (float): Saldo inicial em BRL.
        steps_size (list): Step sizes para os pares.
        pair_prices (list): Lista de dicionários com preços bid/ask para os pares.
        fee_rate (float): Taxa aplicada em cada operação.

    Returns:
        tuple: Lucro estimado e saldo final.
    """
    # Etapa 1: Comprar o ativo do primeiro par (USDT com BRL)
    first_conversion = brl_balance / pair_prices[0]['ask']  # Compra (usa ask)
    first_conversion *= (1 - fee_rate)
    first_conversion = adjust_quantity(first_conversion, steps_size[0])

    # Etapa 2: Comprar o ativo do segundo par
    second_conversion = first_conversion / pair_prices[1]['ask']  # Compra (usa ask)
    second_conversion *= (1 - fee_rate)
    second_conversion = adjust_quantity(second_conversion, steps_size[1])

    # Etapa 3: Vender o ativo do terceiro par para BRL
    final_brl = second_conversion * pair_prices[2]['bid']  # Venda (usa bid)
    final_brl *= (1 - fee_rate)
    final_brl = adjust_quantity(final_brl, steps_size[2])

    # Lucro estimado
    profit = final_brl - brl_balance
    return profit, final_brl


# Lista de pares de arbitragem
arbitrage_opportunities = [
    ('USDTBRL', 'POLUSDT', 'POLBRL'),
    ('USDTBRL', 'SOLUSDT', 'SOLBRL'),
    ('USDTBRL', 'LINKUSDT', 'LINKBRL'),
    ('USDTBRL', 'XRPUSDT', 'XRPBRL'),
    ('USDTBRL', 'DOGEUSDT', 'DOGEBRL'),
    ('USDTBRL', 'NEARUSDT', 'NEARBRL'),
    ('USDTBRL', 'ADAUSDT', 'ADABRL'),
    ('USDTBRL', 'SHIBUSDT', 'SHIBBRL'),
    ('USDTBRL', 'BNBUSDT', 'BNBBRL'),
    ('USDTBRL', 'DOTUSDT', 'DOTBRL'),
    ('USDTBRL', 'PEPEUSDT', 'PEPEBRL'),
    ##('USDTBRL', 'RENDERUSDT', 'RENDERBRL'),
    ##('USDTBRL', 'ENAUSDT', 'ENABRL'),
    ###('USDTBRL', 'CATIUSDT', 'CATIBRL'),
    ####('USDTBRL', 'NOTUSDT', 'NOTBRL'),
    ####('USDTBRL', 'SUIUSDT', 'SUIBRL'),
    ####('USDTBRL', 'FETUSDT', 'FETBRL'),
    ############## ('USDTBRL', 'GALAUSDT', 'GALABRL'),
    ###('USDTBRL', 'NEIROUSDT', 'NEIROBRL'),
    ###('USDTBRL', 'PNUTUSDT', 'PNUTBRL'),
]

# Saldo inicial em BRL
brl_balance = 200

while True:
    # Itera sobre cada combinação de pares na lista
    for opportunity in arbitrage_opportunities:
        try:
            
            # Obter lot sizes para cada par
            lot_size1 = get_lot_size(opportunity[0])
            lot_size2 = get_lot_size(opportunity[1])
            lot_size3 = get_lot_size(opportunity[2])

            if not lot_size1 or not lot_size2 or not lot_size3:
                print(f"Erro: Não foi possível obter lot sizes para os pares {opportunity}.")
                break
        
            # Extrair step sizes
            steps_size = []
            steps_size.append(lot_size1['stepSize'])
            steps_size.append(lot_size2['stepSize'])
            steps_size.append(lot_size3['stepSize'])

            # Obter preços bid/ask para os pares
            prices = fetch_prices_with_best(opportunity)
            
            if all(prices.values()):  # Certifica-se de que todos os preços foram obtidos
                pair_prices = [
                    prices[opportunity[0]],
                    prices[opportunity[1]],
                    prices[opportunity[2]]
                ]
        
                # Calcula o lucro potencial ajustado pelas taxas
                profit, final_usdt = triangular_arbitrage(
                    brl_balance, steps_size, pair_prices
                )

                
                if profit > 1:  # Se há lucro potencial
                    print(f"Lucro: ${profit:.2f}")
                    print(f"Saldo final estimado (após taxas): ${final_usdt:.2f}")

                    # Verifica saldo inicial
                    initial_asset = 'BRL'
                    initial_balance = get_balance(initial_asset)
                    print(f"Esse é o saldo total e o valor investido de BRL: {initial_balance} e {brl_balance}")
                    if initial_balance >= brl_balance-0.5:
                        # Etapa 1: Converter USDT para o ativo do primeiro par
                        symbol1 = opportunity[0]
                        first_conversion_quantity = brl_balance / pair_prices[0]
                        print(f"Esse é o valor convertido de BRL para USDT: {first_conversion_quantity}")
                        first_conversion_quantity = adjust_quantity(first_conversion_quantity, steps_size[0])
                        print(f"Esse é a quantidade USDT atualizado para compra: {first_conversion_quantity}")
                        
                        print("Comprando primeiro ativo.....")
                        execute_order('BUY', symbol1, first_conversion_quantity)
                        print("Comprado!!!!!")
                        
                        print(" ")
                        # print("Descansa meio segundo apos primeira compra.....................")
                        # time.sleep(0.5)
                        print(" ")
                        
                        # Etapa 2: Converter ativo do segundo par
                        second_asset = 'USDT'
                        second_balance = get_balance(second_asset)
                        print(f"Esse é o valor atualizado de USDT: {second_balance}")
                        if second_balance >= first_conversion_quantity-0.5:
                            symbol2 = opportunity[1]
                            second_quantity = second_balance/pair_prices[1]
                            print(f"Esse é a quantidade de {symbol2} que sera comprado: {second_quantity}")
                            second_quantity_adjusted = adjust_quantity(second_quantity*(1-0.001), steps_size[1])
                            print(f"Esse é a quantidade ajustada de {symbol2} que sera comprado: {second_quantity_adjusted}")
                            
                            print("Comprando segundo ativo.....")
                            execute_order('BUY', symbol2, second_quantity_adjusted)
                            print("Comprado!!!!!")
                            
                            print(" ")
                            # print("Descansa meio segundo apos segunda compra.....................")
                            # time.sleep(0.5)
                            print(" ")
                            
                            # Etapa 3: Converter ativo do terceiro par para USDT
                            third_asset = symbol2[:-4]
                            third_balance = get_balance(third_asset)
                            print(f"Esse é a quantidade de {third_asset} que tem no saldo: {third_balance}")
                            if third_balance > 1:
                                symbol3 = opportunity[2]
                                third_balance = adjust_quantity(third_balance, steps_size[2])
                                print(f"Esse é a quantidade {third_balance} comprado para: {symbol3}")

                                print("Comprando terceiro ativo.....")
                                execute_order('SELL', symbol3, third_balance)
                                print("Comprado!!!!")

                                print(" ")
                                print(f"Arbitragem completa para {opportunity}.")

                                break
                            else:
                                print(f"Saldo insuficiente para a etapa final: {third_asset}")
                                break

                        else:
                            print(f"Saldo insuficiente para o segundo ativo: {second_asset}")
                            break
                    else:
                        print("Saldo inicial insuficiente para começar a arbitragem.")
                        break
                
                # else:
                #     print(f"Sem oportunidades para {opportunity}. Lucro: ${profit:.2f}")

            else:
                print(f"Erro ao obter preços para os pares {opportunity}.")
        except Exception as e:
            print(f"Erro ao processar a arbitragem para {opportunity}: {e}")
    
    else:
        # Espera antes de buscar preços novamente
        time.sleep(1)
        # Este bloco else será executado se nenhum break for acionado no loop for
        continue

    # Se o break for acionado no loop for, ele sai do loop while
    break



In [52]:
# depth = client.get_order_book(symbol='ENAUSDT')
# print(depth)

In [44]:
symbol2[:-4]

'ENA'

In [45]:
third_asset = symbol2[:-4]
get_balance(third_asset)

81.736371

In [48]:
symbol3 = opportunity[2]
print(third_asset, steps_size[2])
adjust_quantity(third_asset, steps_size[2])

ENA 0.01


TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [ ]:
adjust_quantity(first_conversion_quantity, steps_size[0])

In [34]:
steps_size[0]

0.1

In [35]:
second_asset = 'USDT'
ss = get_balance(second_asset)
ss

81.88074736

In [36]:
adjust_quantity(ss, steps_size[0])

81.80000000000001

In [135]:
def get_lot_size(symbol):
    """Obtém os detalhes do filtro LOT_SIZE para o par."""
    try:
        exchange_info = client.get_exchange_info()
        for s in exchange_info['symbols']:
            if s['symbol'] == symbol:
                for f in s['filters']:
                    if f['filterType'] == 'LOT_SIZE':
                        return {
                            "minQty": float(f['minQty']),
                            "maxQty": float(f['maxQty']),
                            "stepSize": float(f['stepSize'])
                        }
    except Exception as e:
        print(f"Erro ao obter LOT_SIZE para {symbol}: {e}")
    return None

# Exemplo de uso
lot_size = get_lot_size('USDTBRL')
print(f"LOT_SIZE para USDTBRL: {lot_size}")


LOT_SIZE para USDTBRL: {'minQty': 0.1, 'maxQty': 9222449.0, 'stepSize': 0.1}


In [123]:
def get_balance(asset):
    """Obtém o saldo atual de um ativo."""
    try:
        account_info = client.get_account()
        balances = {item['asset']: float(item['free']) for item in account_info['balances']}
        return balances.get(asset, 0.0)
    except Exception as e:
        print(f"Erro ao obter saldo de {asset}: {e}")
        return 0.0


In [124]:
def execute_order(order_type, symbol, quantity):
    """
    Executa uma ordem de mercado.
    Args:
        order_type (str): 'BUY' ou 'SELL'.
        symbol (str): Par de negociação (ex.: 'BTCUSDT').
        quantity (float): Quantidade do ativo a negociar.
    Returns:
        dict: Detalhes da ordem executada ou None em caso de erro.
    """
    try:
        if order_type == 'BUY':
            order = client.order_market_buy(symbol=symbol, quantity=quantity)
        elif order_type == 'SELL':
            order = client.order_market_sell(symbol=symbol, quantity=quantity)
        else:
            raise ValueError("Tipo de ordem inválido.")
        print(f"Ordem executada: {order}")
        return order
    except Exception as e:
        print(f"Erro ao executar ordem {order_type} para {symbol}: {e}")
        return None


In [125]:
opportunity = arbitrage_opportunities[0]
opportunity

('USDTBRL', 'ENAUSDT', 'ENABRL')

In [126]:
prices = fetch_prices(arbitrage_opportunities[0])
pair1_price = prices[opportunity[0]]
pair2_price = prices[opportunity[1]]
pair3_price = prices[opportunity[2]]

In [127]:
profit, final_usdt = triangular_arbitrage(
    brl_balance, pair1_price, pair2_price, pair3_price
)

In [128]:
profit, final_usdt

(-0.17991999999999564, 99.82008)

In [129]:
# Verifica saldo inicial
initial_asset = 'BRL'
initial_balance = get_balance(initial_asset)
initial_balance

979.64541772

In [130]:
symbol1 = opportunity[0]
symbol1

'USDTBRL'

In [131]:

first_conversion_quantity = brl_balance / pair1_price
symbol1, first_conversion_quantity

('USDTBRL', 16.420361247947454)

In [134]:
execute_order('BUY', symbol1, round(first_conversion_quantity, 4))

Erro ao executar ordem BUY para USDTBRL: APIError(code=-1013): Filter failure: LOT_SIZE


In [40]:
# Etapa 2: Converter ativo do segundo par
second_asset = 'USDT'
second_balance = get_balance(second_asset)
second_asset, second_balance

('USDT', 16.48311736)

In [62]:
quantity2 = second_balance/pair2_price
quantity2

2.0175174247246024

In [104]:
symbol2 = opportunity[1]
symbol2

'RENDERUSDT'

In [65]:
if second_balance >= first_conversion_quantity:
    symbol2 = opportunity[1]
    execute_order('BUY', symbol2, round(quantity2,2))

Ordem executada: {'symbol': 'RENDERUSDT', 'orderId': 381743119, 'orderListId': -1, 'clientOrderId': '80hTed6GOWbp2F1GCfBQBy', 'transactTime': 1733854367596, 'price': '0.00000000', 'origQty': '2.02000000', 'executedQty': '2.02000000', 'cummulativeQuoteQty': '16.42866000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1733854367596, 'fills': [{'price': '8.13300000', 'qty': '2.02000000', 'commission': '0.00202000', 'commissionAsset': 'RENDER', 'tradeId': 21135125}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}


In [69]:
# Etapa 3: Converter ativo do terceiro par para BRL
third_asset = "RENDER"
third_balance = get_balance(third_asset)
symbol3 = opportunity[2]
third_asset, third_balance, symbol3

('RENDER', 2.024765, 'RENDERBRL')

In [73]:

if third_balance > 0:
    symbol3 = opportunity[2]
    execute_order('SELL', symbol3, round(third_balance, 2))
    print(f"Arbitragem completa para {opportunity}.")

Ordem executada: {'symbol': 'RENDERBRL', 'orderId': 9501516, 'orderListId': -1, 'clientOrderId': '4Uac9qHd0T3ovVscEwgJXm', 'transactTime': 1733854633715, 'price': '0.00000000', 'origQty': '2.02000000', 'executedQty': '2.02000000', 'cummulativeQuoteQty': '100.91920000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'workingTime': 1733854633715, 'fills': [{'price': '49.96000000', 'qty': '2.02000000', 'commission': '0.10091920', 'commissionAsset': 'BRL', 'tradeId': 63973}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}
Arbitragem completa para ('USDTBRL', 'RENDERUSDT', 'RENDERBRL').
